# Predicting the Sentiment of IMDB Movie Reviews using LSTM in PyTorch
> This is a practice notebook to work with a dataset of 50,000 movie reviews from the Internet Movie Database (IMDB) and build an LSTM predictor to distinguish between positive and negative reviews.

- toc: true 
- badges: true
- comments: true
- categories: [pytorch, lstm]
- keyword: [ml, dl, nn, pytorch, LSTM, IMDB, sentiment]
- image: images/copied_from_nb/images/2022-11-09-pytorch-lstm-imdb-sentiment-prediction.jpeg

![](images/2022-11-09-pytorch-lstm-imdb-sentiment-prediction.jpeg)

## Credits
This notebook takes inspiration and ideas from the following sources.

* "Machine learning with PyTorch and Scikit-Learn" by "Sebastian Raschka, Yuxi (Hayden) Liu, and Vahid Mirjalili". You can get the book from its website: [Machine learning with PyTorch and Scikit-Learn](https://sebastianraschka.com/books/#machine-learning-with-pytorch-and-scikit-learn). In addition, the GitHub repository for this book has valuable notebooks: [github.com/rasbt/machine-learning-book](https://github.com/rasbt/machine-learning-book). Parts of the code you see in this notebook are taken from [chapter 15](https://github.com/rasbt/machine-learning-book/blob/main/ch15/ch15_part2.ipynb) notebook of the same book.
* "Intro to Deep Learning and Generative Models Course" lecture series from "Sebastian Raschka". Course website: [stat453-ss2021](https://sebastianraschka.com/teaching/stat453-ss2021/). YouTube Link: [Intro to Deep Learning and Generative Models Course](https://www.youtube.com/playlist?list=PLTKMiZHVd_2KJtIXOW0zFhFfBaJJilH51). Lectures that are related to this post are [L15.5 Long Short-Term Memory](https://youtu.be/k6fSgUaWUF8) and [L15.7 An RNN Sentiment Classifier in PyTorch](https://youtu.be/KgrdifrlDxg)

## Environment
This notebook is prepared with Google Colab.

In [1]:
#collapse
from platform import python_version
import numpy, matplotlib, pandas, torch, seaborn

print("python==" + python_version())
print("numpy==" + numpy.__version__)
print("torch==" + torch.__version__)
print("matplotlib==" + matplotlib.__version__)

python==3.7.15
numpy==1.21.6
torch==1.12.1+cu113
matplotlib==3.2.2
seaborn==0.11.2


## Data preparation